In [3]:
from pyspark import SparkContext, SparkConf, SparkFiles

In [4]:
# arrange the data
def mapper1(line):
    node = line.split("\t")
    return node[0], node[1]


In [5]:
# for initial one only
def mapper2(line):
    return line[0], (line[1], initial_rank)


In [6]:
# calculate every rnew'
def mapper3(line):
    # number of outlinks
    cnt = len(line[1][0])  
    
    maplist = []
    
    # to keep the nodes with outlink but no inlink
    maplist.append((line[0], 0))
        
    for des in line[1][0]:
        tmp = beta * (line[1][1]/cnt)
        maplist.append((des, tmp))
            
    return maplist


In [7]:
# calculate rnew
def mapper4(line):
    tmp = line[1] + fix
    return line[0], tmp

In [8]:
#Sum up all rnew's
def reducer1(x, y):
    return x+y

In [9]:
path = "p2p-Gnutella04.txt"
file = sc.textFile(path)
total_node = 10876
initial_rank = 1/total_node
beta = 0.8
s, fix = 0, 0

map1 = file.map(mapper1).groupByKey().mapValues(list)
record = map1.map(mapper2)

for i in range(20):
    r_new_p = record.flatMap(mapper3)
    r_new_p_sum = r_new_p.reduceByKey(reducer1)
    total_node = r_new_p_sum.count()

    # calculate 1-s/N
    s = r_new_p_sum.values().sum()
    fix = (1-s)/total_node
    
    rank = r_new_p_sum.map(mapper4)
    record = map1.join(rank)

rank = rank.takeOrdered(10, key=lambda x: -x[1])
    
outfile = open("Outputfile.txt", "w")
for ans in rank:
    print(ans[0], '%.6f'%ans[1])
    result = str(ans[0]) + "\t" + str('%.6f'%ans[1]) + "\n"
    outfile.write(result)
    
outfile.close()

Py4JJavaError: An error occurred while calling o32.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/Users/dorawang/Desktop/大學/大三上/巨量資料/Homework/HW2/MDA_HW2_107062103/p2p-Gnutella04.txt
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:205)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:276)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:272)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:276)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:272)
	at org.apache.spark.api.

In [8]:
sc.stop()

# MDA_HW2 PageRank

這次的作業要計算在給定的graph中，經過20個iteraiton後各個node的rank，最後列出排名前十高的node和他們相對應的rank。
根據計算的公式

$r^{new} = r'^{new} + \frac{1-S}{N}, \; 其中 \; r'^{new} = \sum \beta \frac{r_i}{d_i}, \; S = \sum r'^{new} \;$ 重複執行20次後，可以得出結果。


在作業中，我總共使用了4個mapper function和1個reducer function。

<br>

* <font color=blue size=4> mapper1 </font>  

    mapper1的主要功能是將讀進來的檔案，以一行為單位整理成（fromID, toID）的key-value pair形式。      
       
<br>
整理完之後，會使用$ groupByKey() \ 和 \ mapValues(list)$把資料再處理，相同的key會group在一起，且讓values合併成一個list。

經過這個操作之後資料會成為$( fromId, [\ toID0,\ toID1,\ toID2, ...] \ )$的格式。
       
<br>
  
* <font color=blue size=4> mapper2 </font>

    mapper2只有在一開始要初始每個outnode的initial rank時會使用到。
    
    資料是以outnode為key，在它的value中再多加入本身的rank這個資訊，方便後續計算時可以方便取來使用。
    
    initial rank是我們給定的，大小就是所有valid node平分，為$ \frac{1}{N}$，作業中的N（total node）有10876個。
    
    所以經過mappe2後，資料會變成 $(fromID, ( \ [ \ toID0,\ toID1,\ toID2, ...], rank ) \ )$。    
<br> 

做好基本的處理之後，就可以開始進行計算的iteration。以下的function在每次的iteration都會執行到：
       
<br>
 
* <font color=blue size=4> mapper3 </font>

    mapper3是計算 $ r'^{new}$ 的部分。
    
    對於每個不同的outnode（key），都會藉由計算$[\ toID0,\ toID1,\ toID2, ...]$  這個list的長度，來獲得他的outlink個數，
    
    再接著以公式算出他對指到的node的貢獻度為多少。
    
    這裡會使用$flatMap$來做map的動作，以分開各個被指到的node。
    
    會先設定一個空的list，用來存放新的資料。
    
    對於每個被指到的node，分別以公式計算從outnode所貢獻出來的rank，用$(node, rank)$這樣的形式存下來，append到list中。
    
    那為了避免發生有些node只有outlink但卻沒有inlink，在後續的計算中被去除掉，導致出現錯誤的這種情形出現，
    
    會幫outnode也建立$(node, rank)$pair，不過將rank設成0，也append到list中。
    
    最後把list return回去。
       
<br>

* <font color=blue size=4> reducer1 </font>

    經過mapper3計算後，會得到所有從outlink那邊所得到的rank的部分（即 $r'^{new}, 也就是\sum \beta \frac{r_i}{d_i}, $ 之中的每一項），

    因為資料結構是$(node, rank)$，key就是被指到的node，因此使用 $reduceByKey$，在reduce1中把所有的rank加總起來，得到完整的$r'^{new}$。
       
<br>

reduce完之後，會重新count一下total node的數量（N）以確保計算正確。

有了所有的$r'^{new}$，我們就可以計算S，進而算出為了修正dead end所要加入的 $\frac{1-S}{N}$。
       
<br>

* <font color=blue size=4> mapper4 </font>

    把算出來的$\frac{1-S}{N}$加到$r'^{new}$中，算出真正的$r^{new}$，同樣以$(node, rank)$的形式return。
        
<br>
   
為了讓每次的iteration能夠以同樣的方式進行計算，最後會用join把算出來新的rank join到原本的node資料之中，也就是$(node, ( \ [ \ toID0,\ toID1,\ toID2, ...], rank ) \ )$這個樣子。

接著就能進行下一次的iteration。



#### Reference:
* https://spark.apache.org/docs/1.1.1/api/python/pyspark.rdd.RDD-class.html#intersection
* https://blog.maxkit.com.tw/2020/02/markdown.html

